# Section 1
## make the table with preparation information

In [1]:
## Now need to add solution treatment, substrate_pretreat and post process info

# 1 if treatment is done and 0 if no treatment

#adding the hole mobility information. only adding hole_mobility not NAN. getting rid of NAN


import pandas as pd
import psycopg2

pgparams = {
    "host": "chbe-ofet-db.postgres.database.azure.com",
    "database": "ofetdb_v2",
    "user": "mg200_ofetdb",
    "password": "MGEROFETDB23!",
    "port": "5432",
}

def read_select_query(query):
    with psycopg2.connect(**pgparams) as conn:
        df = pd.read_sql_query(query, conn)
    return df

# SQL query to fetch the required data
query ='''
    SELECT
        s.sample_id,
        e.citation_type,
        e.meta as experiment_meta,
        sol.concentration as solution_concentration,
        sm_s.vol_frac as solvent_vol_frac,
        sv.pubchem_cid as solvent_pubchem_cid,
        sv.iupac_name as solvent_iupac_name,
        p.common_name as polymer_common_name,
        p.iupac_name as polymer_iupac_name,
        p.mw as polymer_mw,
        p.mn as polymer_mn,
        p.dispersity as polymer_dispersity,
        df.params as device_substrate_parameters,
        fd.deposition_type as film_deposition_type,
        fd.params as film_deposition_params,
        CAST(m.data -> 'hole_mobility' ->> 'value' AS FLOAT) AS hole_mobility,
        CASE WHEN op.solution_treatment_id IS NOT NULL THEN 1 ELSE 0 END AS Solution_treatment,
        CASE WHEN op.substrate_pretreat_id IS NOT NULL THEN 1 ELSE 0 END AS Substrate_pretreatment,
        CASE WHEN op.postprocess_id IS NOT NULL THEN 1 ELSE 0 END AS Post_Process
    FROM
        SAMPLE s
    JOIN
        EXPERIMENT_INFO e ON s.exp_id = e.exp_id
    LEFT JOIN
        OFET_PROCESS op ON s.process_id = op.process_id
    LEFT JOIN
        SOLUTION sol ON op.solution_id = sol.solution_id
    LEFT JOIN
        SOLUTION_MAKEUP_SOLVENT sm_s ON sol.solution_id = sm_s.solution_id
    LEFT JOIN
        SOLVENT sv ON sm_s.solvent_id = sv.pubchem_cid
    LEFT JOIN
        SOLUTION_MAKEUP_POLYMER sm_p ON sol.solution_id = sm_p.solution_id
    LEFT JOIN
        POLYMER p ON sm_p.polymer_id = p.polymer_id
    LEFT JOIN
        DEVICE_FABRICATION df ON op.device_fab_id = df.device_fab_id
    LEFT JOIN
        FILM_DEPOSITION fd ON op.film_deposition_id = fd.film_deposition_id
    LEFT JOIN
        MEASUREMENT m ON s.sample_id = m.sample_id
    WHERE
        sm_s.vol_frac = 1 AND
        sm_p.wt_frac = 1 AND
        m.measurement_type = 'transfer_curve' AND
        m.data -> 'hole_mobility' ->> 'value' IS NOT NULL;
'''


# Use the read_select_query function to execute the query
result_df = read_select_query(query)

# Display the resulting DataFrame
result_df

C:\Users\Myeongyeon Lee\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,sample_id,citation_type,experiment_meta,solution_concentration,solvent_vol_frac,solvent_pubchem_cid,solvent_iupac_name,polymer_common_name,polymer_iupac_name,polymer_mw,polymer_mn,polymer_dispersity,device_substrate_parameters,film_deposition_type,film_deposition_params,hole_mobility,solution_treatment,substrate_pretreatment,post_process
0,1,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.0,1.0,7964,chlorobenzene,DPP-DTT,"poly[2,5-(2-octyldodecyl)-3,6-diketopyrrolopyr...",299.0,90.0,3.32,"{'channel_width': 1500.0, 'gate_material': 'n-...",spin,"{'spin_rate': 1000.0, 'environment': 'air'}",0.11,0,0,1
1,2,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.0,1.0,7964,chlorobenzene,DPP-DTT,"poly[2,5-(2-octyldodecyl)-3,6-diketopyrrolopyr...",299.0,90.0,3.32,"{'channel_width': 1500.0, 'gate_material': 'n-...",spin,"{'spin_rate': 1000.0, 'environment': 'air'}",0.29,0,1,1
2,3,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.0,1.0,7964,chlorobenzene,DPP-DTT,"poly[2,5-(2-octyldodecyl)-3,6-diketopyrrolopyr...",299.0,90.0,3.32,"{'channel_width': 1500.0, 'gate_material': 'n-...",spin,"{'spin_rate': 1000.0, 'environment': 'air'}",0.23,0,1,1
3,4,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.0,1.0,7964,chlorobenzene,DPP-DTT,"poly[2,5-(2-octyldodecyl)-3,6-diketopyrrolopyr...",299.0,90.0,3.32,"{'channel_width': 1500.0, 'gate_material': 'n-...",spin,"{'spin_rate': 1000.0, 'environment': 'air'}",0.73,0,1,1
4,5,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.0,1.0,7964,chlorobenzene,DPP-DTT,"poly[2,5-(2-octyldodecyl)-3,6-diketopyrrolopyr...",299.0,90.0,3.32,"{'channel_width': 1500.0, 'gate_material': 'n-...",spin,"{'spin_rate': 1000.0, 'environment': 'air'}",1.86,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,868,literature,"{'ID': 'Chang_2016', 'doi': '10.1021/acsami.6b...",5.0,1.0,6212,chloroform,P3HT,"poly(3-hexylthiophene-2,5-diyl)",90.0,NaN,NaN,"{'channel_width': 2000, 'gate_material': 'n-do...",spin,"{'spin_rate': 1500, 'spin_time': 60, 'environm...",0.10,1,1,1
510,869,literature,"{'ID': 'Chang_2016', 'doi': '10.1021/acsami.6b...",5.0,1.0,6212,chloroform,P3HT,"poly(3-hexylthiophene-2,5-diyl)",90.0,NaN,NaN,"{'channel_width': 2000, 'gate_material': 'n-do...",shear,"{'environment': 'air', 'temperature': 25, 'coa...",0.10,1,1,1
511,870,literature,"{'ID': 'Chang_2016', 'doi': '10.1021/acsami.6b...",5.0,1.0,6212,chloroform,P3HT,"poly(3-hexylthiophene-2,5-diyl)",90.0,NaN,NaN,"{'channel_width': 2000, 'gate_material': 'n-do...",shear,"{'environment': 'air', 'temperature': 25, 'coa...",0.20,1,1,1
512,871,literature,"{'ID': 'Chang_2016', 'doi': '10.1021/acsami.6b...",5.0,1.0,6212,chloroform,P3HT,"poly(3-hexylthiophene-2,5-diyl)",90.0,NaN,NaN,"{'channel_width': 2000, 'gate_material': 'n-do...",shear,"{'environment': 'air', 'temperature': 25, 'coa...",0.13,1,1,1


In [2]:
 #unpacking the information stored in device_substrate_parameters

import pandas as pd
from pandas import json_normalize



# # Extract the 'device_substrate_parameters' column and normalize it
params_df = json_normalize(result_df['device_substrate_parameters'])

# # Concatenate the original DataFrame with the new 'params_df'
result_df = pd.concat([result_df, params_df], axis=1)

# # Drop the original 'device_substrate_parameters' column
result_df = result_df.drop('device_substrate_parameters', axis=1)

# # Display the resulting DataFrame
result_df

,sample_id,citation_type,experiment_meta,solution_concentration,solvent_vol_frac,solvent_pubchem_cid,solvent_iupac_name,polymer_common_name,polymer_iupac_name,polymer_mw,...,gate_material,channel_length,dielectric_material,dielectric_thickness,electrode_configuration,dielectric_material_2,dielectric_thickness_2,dielectric_1_material,dielectric_1_thickness,substrate_material
0,1,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.0,1.0,7964,chlorobenzene,DPP-DTT,"poly[2,5-(2-octyldodecyl)-3,6-diketopyrrolopyr...",299.0,...,n-doped Si,80.0,SiO2,300.0,BGTC,NaN,NaN,NaN,NaN,NaN
1,2,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.0,1.0,7964,chlorobenzene,DPP-DTT,"poly[2,5-(2-octyldodecyl)-3,6-diketopyrrolopyr...",299.0,...,n-doped Si,80.0,SiO2,300.0,BGTC,NaN,NaN,NaN,NaN,NaN
2,3,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.0,1.0,7964,chlorobenzene,DPP-DTT,"poly[2,5-(2-octyldodecyl)-3,6-diketopyrrolopyr...",299.0,...,n-doped Si,80.0,SiO2,300.0,BGTC,NaN,NaN,NaN,NaN,NaN
3,4,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.0,1.0,7964,chlorobenzene,DPP-DTT,"poly[2,5-(2-octyldodecyl)-3,6-diketopyrrolopyr...",299.0,...,n-doped Si,80.0,SiO2,300.0,BGTC,NaN,NaN,NaN,NaN,NaN
4,5,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.0,1.0,7964,chlorobenzene,DPP-DTT,"poly[2,5-(2-octyldodecyl)-3,6-diketopyrrolopyr...",299.0,...,n-doped Si,80.0,SiO2,300.0,BGTC,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,868,literature,"{'ID': 'Chang_2016', 'doi': '10.1021/acsami.6b...",5.0,1.0,6212,chloroform,P3HT,"poly(3-hexylthiophene-2,5-diyl)",90.0,...,n-doped Si,50.0,NaN,NaN,BGBC,NaN,NaN,SiO2,300.0,NaN
510,869,literature,"{'ID': 'Chang_2016', 'doi': '10.1021/acsami.6b...",5.0,1.0,6212,chloroform,P3HT,"poly(3-hexylthiophene-2,5-diyl)",90.0,...,n-doped Si,50.0,NaN,NaN,BGBC,NaN,NaN,SiO2,300.0,NaN
511,870,literature,"{'ID': 'Chang_2016', 'doi': '10.1021/acsami.6b...",5.0,1.0,6212,chloroform,P3HT,"poly(3-hexylthiophene-2,5-diyl)",90.0,...,n-doped Si,50.0,NaN,NaN,BGBC,NaN,NaN,SiO2,300.0,NaN
512,871,literature,"{'ID': 'Chang_2016', 'doi': '10.1021/acsami.6b...",5.0,1.0,6212,chloroform,P3HT,"poly(3-hexylthiophene-2,5-diyl)",90.0,...,n-doped Si,50.0,NaN,NaN,BGBC,NaN,NaN,SiO2,300.0,NaN


In [3]:

# Drop unnecessary columns that won't be used for modeling
columns_to_drop = ['film_deposition_params', 'citation_type', 'experiment_meta', 'solvent_vol_frac', 'solvent_iupac_name', 'polymer_iupac_name', 'dielectric_material_2', 'dielectric_thickness_2', 'dielectric_1_material', 'dielectric_1_thickness', 'substrate_material']
result_df = result_df.drop(columns=columns_to_drop)

# Display the resulting DataFrame
result_df


,sample_id,solution_concentration,solvent_pubchem_cid,polymer_common_name,polymer_mw,polymer_mn,polymer_dispersity,film_deposition_type,hole_mobility,solution_treatment,substrate_pretreatment,post_process,channel_width,gate_material,channel_length,dielectric_material,dielectric_thickness,electrode_configuration
0,1,4.0,7964,DPP-DTT,299.0,90.0,3.32,spin,0.11,0,0,1,1500.0,n-doped Si,80.0,SiO2,300.0,BGTC
1,2,4.0,7964,DPP-DTT,299.0,90.0,3.32,spin,0.29,0,1,1,1500.0,n-doped Si,80.0,SiO2,300.0,BGTC
2,3,4.0,7964,DPP-DTT,299.0,90.0,3.32,spin,0.23,0,1,1,1500.0,n-doped Si,80.0,SiO2,300.0,BGTC
3,4,4.0,7964,DPP-DTT,299.0,90.0,3.32,spin,0.73,0,1,1,1500.0,n-doped Si,80.0,SiO2,300.0,BGTC
4,5,4.0,7964,DPP-DTT,299.0,90.0,3.32,spin,1.86,0,1,1,1500.0,n-doped Si,80.0,SiO2,300.0,BGTC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,868,5.0,6212,P3HT,90.0,NaN,NaN,spin,0.10,1,1,1,2000.0,n-doped Si,50.0,NaN,NaN,BGBC
510,869,5.0,6212,P3HT,90.0,NaN,NaN,shear,0.10,1,1,1,2000.0,n-doped Si,50.0,NaN,NaN,BGBC
511,870,5.0,6212,P3HT,90.0,NaN,NaN,shear,0.20,1,1,1,2000.0,n-doped Si,50.0,NaN,NaN,BGBC
512,871,5.0,6212,P3HT,90.0,NaN,NaN,shear,0.13,1,1,1,2000.0,n-doped Si,50.0,NaN,NaN,BGBC


In [4]:
# List of columns to one-hot encode
columns_to_one_hot_encode = ['film_deposition_type', 'gate_material', 'dielectric_material', 'electrode_configuration']

# Perform one-hot encoding
result_df = pd.get_dummies(result_df, columns=columns_to_one_hot_encode)

# Display the resulting DataFrame with one-hot encoding
result_df


,sample_id,solution_concentration,solvent_pubchem_cid,polymer_common_name,polymer_mw,polymer_mn,polymer_dispersity,hole_mobility,solution_treatment,substrate_pretreatment,...,dielectric_material_PAN,dielectric_material_PMMA,dielectric_material_PTrFE,dielectric_material_PVP,dielectric_material_Shellac,dielectric_material_Si3N4,dielectric_material_SiO2,electrode_configuration_BGBC,electrode_configuration_BGTC,electrode_configuration_TGBC
0,1,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.11,0,0,...,0,0,0,0,0,0,1,0,1,0
1,2,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.29,0,1,...,0,0,0,0,0,0,1,0,1,0
2,3,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.23,0,1,...,0,0,0,0,0,0,1,0,1,0
3,4,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.73,0,1,...,0,0,0,0,0,0,1,0,1,0
4,5,4.0,7964,DPP-DTT,299.0,90.0,3.32,1.86,0,1,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,868,5.0,6212,P3HT,90.0,NaN,NaN,0.10,1,1,...,0,0,0,0,0,0,0,1,0,0
510,869,5.0,6212,P3HT,90.0,NaN,NaN,0.10,1,1,...,0,0,0,0,0,0,0,1,0,0
511,870,5.0,6212,P3HT,90.0,NaN,NaN,0.20,1,1,...,0,0,0,0,0,0,0,1,0,0
512,871,5.0,6212,P3HT,90.0,NaN,NaN,0.13,1,1,...,0,0,0,0,0,0,0,1,0,0


In [5]:
column_names = result_df.columns
print(column_names)


Index(['sample_id', 'solution_concentration', 'solvent_pubchem_cid',
       'polymer_common_name', 'polymer_mw', 'polymer_mn', 'polymer_dispersity',
       'hole_mobility', 'solution_treatment', 'substrate_pretreatment',
       'post_process', 'channel_width', 'channel_length',
       'dielectric_thickness', 'film_deposition_type_Dip Coating',
       'film_deposition_type_blade', 'film_deposition_type_dip',
       'film_deposition_type_drop', 'film_deposition_type_inkjet',
       'film_deposition_type_shear', 'film_deposition_type_spin',
       'film_deposition_type_wire', 'gate_material_Al', 'gate_material_Au',
       'gate_material_PEDOT:PSS', 'gate_material_PET', 'gate_material_Si',
       'gate_material_glass', 'gate_material_n-doped Si',
       'gate_material_p-doped Si', 'dielectric_material_6FDA-DABC',
       'dielectric_material_CYTOP', 'dielectric_material_PAN',
       'dielectric_material_PMMA', 'dielectric_material_PTrFE',
       'dielectric_material_PVP', 'dielectric_materi

In [6]:

#consolidating gate material columns
gate_material_Si_to_consolidate = ['gate_material_n-doped Si', 'gate_material_Si','gate_material_p-doped Si']
gate_material_other_to_consolidate = ['gate_material_Al', 'gate_material_Au', 'gate_material_PEDOT:PSS', 'gate_material_PET','gate_material_glass']
# Create new columns
result_df['gate_material_Si'] = result_df[gate_material_Si_to_consolidate].max(axis=1)
result_df['gate_material_Other'] = result_df[gate_material_other_to_consolidate].max(axis=1)
# Drop the original columns
result_df.drop(columns=gate_material_Si_to_consolidate, inplace=True)
result_df.drop(columns=gate_material_other_to_consolidate, inplace=True)


#consolidating coating columns
MGC_columns_to_consolidate = ['film_deposition_type_Dip Coating',
                           'film_deposition_type_blade',
                           'film_deposition_type_dip',
                           'film_deposition_type_inkjet',
                           'film_deposition_type_shear',
                           'film_deposition_type_wire']
# Create new columns
result_df['film_deposition_type_MGC'] = result_df[MGC_columns_to_consolidate].max(axis=1)
# Drop the original columns
result_df.drop(columns=MGC_columns_to_consolidate, inplace=True)


#consolidating dielectric columns
dielectric_material_columns_to_consolidate = ['dielectric_material_6FDA-DABC',
       'dielectric_material_CYTOP', 'dielectric_material_PAN',
       'dielectric_material_PMMA', 'dielectric_material_PTrFE',
       'dielectric_material_PVP', 'dielectric_material_Shellac',
       'dielectric_material_Si3N4']
# Create new columns
result_df['dielectric_material_other'] = result_df[dielectric_material_columns_to_consolidate].max(axis=1)
# Drop the original columns
result_df.drop(columns=dielectric_material_columns_to_consolidate, inplace=True)


In [7]:
result_df

,sample_id,solution_concentration,solvent_pubchem_cid,polymer_common_name,polymer_mw,polymer_mn,polymer_dispersity,hole_mobility,solution_treatment,substrate_pretreatment,...,dielectric_thickness,film_deposition_type_drop,film_deposition_type_spin,dielectric_material_SiO2,electrode_configuration_BGBC,electrode_configuration_BGTC,electrode_configuration_TGBC,gate_material_Other,film_deposition_type_MGC,dielectric_material_other
0,1,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.11,0,0,...,300.0,0,1,1,0,1,0,0,0,0
1,2,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.29,0,1,...,300.0,0,1,1,0,1,0,0,0,0
2,3,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.23,0,1,...,300.0,0,1,1,0,1,0,0,0,0
3,4,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.73,0,1,...,300.0,0,1,1,0,1,0,0,0,0
4,5,4.0,7964,DPP-DTT,299.0,90.0,3.32,1.86,0,1,...,300.0,0,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,868,5.0,6212,P3HT,90.0,NaN,NaN,0.10,1,1,...,NaN,0,1,0,1,0,0,0,0,0
510,869,5.0,6212,P3HT,90.0,NaN,NaN,0.10,1,1,...,NaN,0,0,0,1,0,0,0,1,0
511,870,5.0,6212,P3HT,90.0,NaN,NaN,0.20,1,1,...,NaN,0,0,0,1,0,0,0,1,0
512,871,5.0,6212,P3HT,90.0,NaN,NaN,0.13,1,1,...,NaN,0,0,0,1,0,0,0,1,0


# Section 2
## make the table with process paramter

In [8]:
##consider processing with order
query = '''
   SELECT
        s.sample_id,
        spstep.params AS params,
        MAX(CASE WHEN spstep.treatment_type = 'sam' THEN 1 ELSE 0 END) AS treatment_type_sam,
        MAX(CASE WHEN spstep.treatment_type = 'plasma_treatment' THEN 1 ELSE 0 END) AS treatment_type_plasma,
        MAX(CASE WHEN spstep.treatment_type = 'uv_ozone' THEN 1 ELSE 0 END) AS treatment_type_uv_ozone,
        
        CASE WHEN spstep.params ->> 'iupac_name' = 'water' THEN 1 ELSE 0 END AS is_water_treatment,
        CASE WHEN spstep.params ->> 'iupac_name' = 'isopropanol' THEN 1 ELSE 0 END AS is_isopropanol_treatment,
        CASE WHEN spstep.params ->> 'iupac_name' = 'ethanol' THEN 1 ELSE 0 END AS is_ethanol_treatment,
        CASE WHEN spstep.params ->> 'iupac_name' = 'acetone' THEN 1 ELSE 0 END AS is_acetone_treatment,
        CASE WHEN spstep.params ->> 'iupac_name' = 'methanol' THEN 1 ELSE 0 END AS is_methanol_treatment,
        CASE WHEN spstep.params ->> 'iupac_name' = 'piranha solution' THEN 1 ELSE 0 END AS is_piranha_treatment,
        CASE WHEN spstep.params ->> 'iupac_name' = 'PEIE' THEN 1 ELSE 0 END AS is_PEIE_treatment,
        CASE WHEN spstep.params ->> 'iupac_name' = 'RCA' THEN 1 ELSE 0 END AS is_RCA_treatment,
        CASE WHEN spstep.params ->> 'iupac_name' = 'toluene' THEN 1 ELSE 0 END AS is_toluene_treatment,
        
        
        MAX(CASE WHEN ststep.treatment_type = 'poor_solvent' THEN 1 ELSE 0 END) AS solution_treatment_poor_solvent,
        MAX(CASE WHEN ststep.treatment_type = 'aging' THEN 1 ELSE 0 END) AS solution_treatment_aging,
        MAX(CASE WHEN ststep.treatment_type = 'sonication' THEN 1 ELSE 0 END) AS solution_treatment_sonication,
        MAX(CASE WHEN ststep.treatment_type = 'mixing' THEN 1 ELSE 0 END) AS solution_treatment_mixing,
        MAX(CASE WHEN ststep.treatment_type = 'uv_irradiation' THEN 1 ELSE 0 END) AS solution_treatment_uv_irradiation,
    
    
        MAX(CASE WHEN ppstep.treatment_type = 'annealing' THEN 1 ELSE 0 END) AS post_process_annealing,
        MAX(CASE WHEN ppstep.treatment_type = 'drying' THEN 1 ELSE 0 END) AS  post_process_drying,
        MAX(CASE WHEN ppstep.treatment_type = 'chemical_treat' THEN 1 ELSE 0 END) AS post_process_chemical
        
       
    -- Add more similar lines for other treatment types as needed
    
    
     FROM
        SAMPLE s
    JOIN
        OFET_PROCESS of ON s.process_id = of.process_id
    LEFT JOIN 
        SUBSTRATE_PRETREAT_ORDER spo ON of.substrate_pretreat_id = spo.substrate_pretreat_id
    LEFT JOIN
        SUBSTRATE_PRETREAT_STEP spstep ON spstep.substrate_pretreat_step_id = spo.substrate_pretreat_step_id
    LEFT JOIN
        SOLUTION_TREATMENT_ORDER sto ON of.solution_treatment_id = sto.solution_treatment_id
    LEFT JOIN
        SOLUTION_TREATMENT_STEP ststep ON sto.solution_treatment_step_id = ststep.solution_treatment_step_id
    LEFT JOIN
        POSTPROCESS_ORDER ppo ON of.postprocess_id = ppo.postprocess_id
    LEFT JOIN
        POSTPROCESS_STEP ppstep ON ppo.postprocess_step_id = ppstep.postprocess_step_id
    GROUP BY
        s.sample_id,
        spstep.params
    ORDER BY
        s.sample_id ASC;

'''


# Use the read_select_query function to execute the query
result_df_chem1= read_select_query(query)

result_df_chem1  

#keep substrate_treatment_type_sam, aging, sonication,uv_irrad,pp_annealing. remove everything else

C:\Users\Myeongyeon Lee\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,sample_id,params,treatment_type_sam,treatment_type_plasma,treatment_type_uv_ozone,is_water_treatment,is_isopropanol_treatment,is_ethanol_treatment,is_acetone_treatment,is_methanol_treatment,...,is_rca_treatment,is_toluene_treatment,solution_treatment_poor_solvent,solution_treatment_aging,solution_treatment_sonication,solution_treatment_mixing,solution_treatment_uv_irradiation,post_process_annealing,post_process_drying,post_process_chemical
0,1,None,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2,"{'sam_name': 'MTS', 'iupac_name': 'methyltrich...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,"{'sam_name': 'PTS', 'iupac_name': 'phenyltrich...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,"{'sam_name': 'OTS-8', 'iupac_name': 'octyltric...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,5,"{'sam_name': 'OTS-18', 'iupac_name': 'octadecy...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832,871,"{'time': 15, 'iupac_name': 'methanol', 'enviro...",0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,1,1,0,0
1833,872,{'time': 15},0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,1,1,0,0
1834,872,"{'time': 15, 'iupac_name': 'acetone', 'environ...",0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,1,1,0,0
1835,872,"{'time': 15, 'iupac_name': 'isopropanol', 'env...",0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,1,1,0,0


In [ ]:
result_df_chem1

In [9]:
result_df_chem1.head(11)

,sample_id,params,treatment_type_sam,treatment_type_plasma,treatment_type_uv_ozone,is_water_treatment,is_isopropanol_treatment,is_ethanol_treatment,is_acetone_treatment,is_methanol_treatment,...,is_rca_treatment,is_toluene_treatment,solution_treatment_poor_solvent,solution_treatment_aging,solution_treatment_sonication,solution_treatment_mixing,solution_treatment_uv_irradiation,post_process_annealing,post_process_drying,post_process_chemical
0,1,None,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2,"{'sam_name': 'MTS', 'iupac_name': 'methyltrich...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,"{'sam_name': 'PTS', 'iupac_name': 'phenyltrich...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,"{'sam_name': 'OTS-8', 'iupac_name': 'octyltric...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,5,"{'sam_name': 'OTS-18', 'iupac_name': 'octadecy...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,6,"{'sam_name': 'MTS', 'iupac_name': 'methyltrich...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,6,"{'sam_name': 'OTS-18', 'iupac_name': 'octadecy...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7,7,"{'sam_name': 'MTS', 'iupac_name': 'methyltrich...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,7,"{'sam_name': 'OTS-18', 'iupac_name': 'octadecy...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,8,"{'sam_name': 'OTS-18', 'iupac_name': 'octadecy...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


# Section 3
## Combining two tables

In [10]:
combined_df = pd.merge(result_df, result_df_chem1, on="sample_id", how="inner")

In [11]:
combined_df

,sample_id,solution_concentration,solvent_pubchem_cid,polymer_common_name,polymer_mw,polymer_mn,polymer_dispersity,hole_mobility,solution_treatment,substrate_pretreatment,...,is_rca_treatment,is_toluene_treatment,solution_treatment_poor_solvent,solution_treatment_aging,solution_treatment_sonication,solution_treatment_mixing,solution_treatment_uv_irradiation,post_process_annealing,post_process_drying,post_process_chemical
0,1,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.11,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.29,0,1,...,0,0,0,0,0,0,0,1,0,0
2,3,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.23,0,1,...,0,0,0,0,0,0,0,1,0,0
3,4,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.73,0,1,...,0,0,0,0,0,0,0,1,0,0
4,5,4.0,7964,DPP-DTT,299.0,90.0,3.32,1.86,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088,871,5.0,6212,P3HT,90.0,NaN,NaN,0.13,1,1,...,0,0,0,0,0,1,1,1,0,0
1089,872,5.0,6212,P3HT,90.0,NaN,NaN,0.09,1,1,...,0,0,0,0,0,1,1,1,0,0
1090,872,5.0,6212,P3HT,90.0,NaN,NaN,0.09,1,1,...,0,0,0,0,0,1,1,1,0,0
1091,872,5.0,6212,P3HT,90.0,NaN,NaN,0.09,1,1,...,0,0,0,0,0,1,1,1,0,0


In [12]:
unique_combined_df = combined_df.drop_duplicates(subset='sample_id')
unique_combined_df

,sample_id,solution_concentration,solvent_pubchem_cid,polymer_common_name,polymer_mw,polymer_mn,polymer_dispersity,hole_mobility,solution_treatment,substrate_pretreatment,...,is_rca_treatment,is_toluene_treatment,solution_treatment_poor_solvent,solution_treatment_aging,solution_treatment_sonication,solution_treatment_mixing,solution_treatment_uv_irradiation,post_process_annealing,post_process_drying,post_process_chemical
0,1,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.11,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.29,0,1,...,0,0,0,0,0,0,0,1,0,0
2,3,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.23,0,1,...,0,0,0,0,0,0,0,1,0,0
3,4,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.73,0,1,...,0,0,0,0,0,0,0,1,0,0
4,5,4.0,7964,DPP-DTT,299.0,90.0,3.32,1.86,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,868,5.0,6212,P3HT,90.0,NaN,NaN,0.10,1,1,...,0,0,0,0,0,1,1,1,0,0
1077,869,5.0,6212,P3HT,90.0,NaN,NaN,0.10,1,1,...,0,0,0,0,0,1,1,1,0,0
1081,870,5.0,6212,P3HT,90.0,NaN,NaN,0.20,1,1,...,0,0,0,0,0,1,1,1,0,0
1085,871,5.0,6212,P3HT,90.0,NaN,NaN,0.13,1,1,...,0,0,0,0,0,1,1,1,0,0


In [13]:
unique_column_names = unique_combined_df.columns
print(unique_column_names)

Index(['sample_id', 'solution_concentration', 'solvent_pubchem_cid',
       'polymer_common_name', 'polymer_mw', 'polymer_mn', 'polymer_dispersity',
       'hole_mobility', 'solution_treatment', 'substrate_pretreatment',
       'post_process', 'channel_width', 'channel_length',
       'dielectric_thickness', 'film_deposition_type_drop',
       'film_deposition_type_spin', 'dielectric_material_SiO2',
       'electrode_configuration_BGBC', 'electrode_configuration_BGTC',
       'electrode_configuration_TGBC', 'gate_material_Other',
       'film_deposition_type_MGC', 'dielectric_material_other', 'params',
       'treatment_type_sam', 'treatment_type_plasma',
       'treatment_type_uv_ozone', 'is_water_treatment',
       'is_isopropanol_treatment', 'is_ethanol_treatment',
       'is_acetone_treatment', 'is_methanol_treatment', 'is_piranha_treatment',
       'is_peie_treatment', 'is_rca_treatment', 'is_toluene_treatment',
       'solution_treatment_poor_solvent', 'solution_treatment_aging',

In [14]:
unique_columns_to_drop = ['params','treatment_type_plasma','treatment_type_uv_ozone', 'is_water_treatment','is_isopropanol_treatment', 'is_ethanol_treatment','is_acetone_treatment', 'is_methanol_treatment', 'is_piranha_treatment','is_peie_treatment', 'is_rca_treatment', 'is_toluene_treatment','solution_treatment_poor_solvent', 'solution_treatment_mixing','post_process_drying', 'post_process_chemical']
unique_combined_df = unique_combined_df.drop(columns=unique_columns_to_drop)
unique_combined_df

,sample_id,solution_concentration,solvent_pubchem_cid,polymer_common_name,polymer_mw,polymer_mn,polymer_dispersity,hole_mobility,solution_treatment,substrate_pretreatment,...,electrode_configuration_BGTC,electrode_configuration_TGBC,gate_material_Other,film_deposition_type_MGC,dielectric_material_other,treatment_type_sam,solution_treatment_aging,solution_treatment_sonication,solution_treatment_uv_irradiation,post_process_annealing
0,1,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.11,0,0,...,1,0,0,0,0,0,0,0,0,1
1,2,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.29,0,1,...,1,0,0,0,0,1,0,0,0,1
2,3,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.23,0,1,...,1,0,0,0,0,1,0,0,0,1
3,4,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.73,0,1,...,1,0,0,0,0,1,0,0,0,1
4,5,4.0,7964,DPP-DTT,299.0,90.0,3.32,1.86,0,1,...,1,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,868,5.0,6212,P3HT,90.0,NaN,NaN,0.10,1,1,...,0,0,0,0,0,0,0,0,1,1
1077,869,5.0,6212,P3HT,90.0,NaN,NaN,0.10,1,1,...,0,0,0,1,0,0,0,0,1,1
1081,870,5.0,6212,P3HT,90.0,NaN,NaN,0.20,1,1,...,0,0,0,1,0,0,0,0,1,1
1085,871,5.0,6212,P3HT,90.0,NaN,NaN,0.13,1,1,...,0,0,0,1,0,0,0,0,1,1


# P3HT Modeling

In [15]:
unique_combined_df_P3HT = unique_combined_df[unique_combined_df['polymer_common_name'] == 'P3HT']
unique_combined_df_P3HT

,sample_id,solution_concentration,solvent_pubchem_cid,polymer_common_name,polymer_mw,polymer_mn,polymer_dispersity,hole_mobility,solution_treatment,substrate_pretreatment,...,electrode_configuration_BGTC,electrode_configuration_TGBC,gate_material_Other,film_deposition_type_MGC,dielectric_material_other,treatment_type_sam,solution_treatment_aging,solution_treatment_sonication,solution_treatment_uv_irradiation,post_process_annealing
162,350,10.0,6212,P3HT,NaN,65.5,NaN,0.00073,0,0,...,0,0,0,0,0,0,0,0,0,0
163,351,10.0,6212,P3HT,NaN,65.5,NaN,0.01560,1,0,...,0,0,0,0,0,0,1,0,0,0
164,352,10.0,6212,P3HT,NaN,37.0,NaN,0.01200,1,1,...,1,0,0,0,0,1,1,0,0,1
165,353,10.0,7809,P3HT,NaN,37.0,NaN,0.04200,1,1,...,1,0,0,0,0,1,1,0,0,1
166,354,10.0,13229,P3HT,NaN,37.0,NaN,0.02200,1,1,...,1,0,0,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,868,5.0,6212,P3HT,90.0,NaN,NaN,0.10000,1,1,...,0,0,0,0,0,0,0,0,1,1
1077,869,5.0,6212,P3HT,90.0,NaN,NaN,0.10000,1,1,...,0,0,0,1,0,0,0,0,1,1
1081,870,5.0,6212,P3HT,90.0,NaN,NaN,0.20000,1,1,...,0,0,0,1,0,0,0,0,1,1
1085,871,5.0,6212,P3HT,90.0,NaN,NaN,0.13000,1,1,...,0,0,0,1,0,0,0,0,1,1


In [16]:
num_rows, num_columns = unique_combined_df_P3HT.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

Number of rows: 327
Number of columns: 28


In [17]:
print(unique_combined_df_P3HT.isna().sum(axis=0))
print('Total observations: {}'.format(unique_combined_df_P3HT.shape[0]))
print('Total features: {}'.format(unique_combined_df_P3HT.shape[1]))

sample_id                              0
solution_concentration                16
solvent_pubchem_cid                    0
polymer_common_name                    0
polymer_mw                            62
polymer_mn                            24
polymer_dispersity                    86
hole_mobility                          0
solution_treatment                     0
substrate_pretreatment                 0
post_process                           0
channel_width                         10
channel_length                        22
dielectric_thickness                 150
film_deposition_type_drop              0
film_deposition_type_spin              0
dielectric_material_SiO2               0
electrode_configuration_BGBC           0
electrode_configuration_BGTC           0
electrode_configuration_TGBC           0
gate_material_Other                    0
film_deposition_type_MGC               0
dielectric_material_other              0
treatment_type_sam                     0
solution_treatme

In [ ]:
# remove columns that have mostly same values or mostly NaN

In [22]:
unique_combined_df_P3HT.to_excel('combined_df_P3HT.xlsx', index=True, header=True)

# DPP-DTT Modeling

In [19]:
unique_combined_df_DPP_DTT = unique_combined_df[unique_combined_df['polymer_common_name'] == 'DPP-DTT']
unique_combined_df_DPP_DTT

,sample_id,solution_concentration,solvent_pubchem_cid,polymer_common_name,polymer_mw,polymer_mn,polymer_dispersity,hole_mobility,solution_treatment,substrate_pretreatment,...,electrode_configuration_BGTC,electrode_configuration_TGBC,gate_material_Other,film_deposition_type_MGC,dielectric_material_other,treatment_type_sam,solution_treatment_aging,solution_treatment_sonication,solution_treatment_uv_irradiation,post_process_annealing
0,1,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.110000,0,0,...,1,0,0,0,0,0,0,0,0,1
1,2,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.290000,0,1,...,1,0,0,0,0,1,0,0,0,1
2,3,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.230000,0,1,...,1,0,0,0,0,1,0,0,0,1
3,4,4.0,7964,DPP-DTT,299.0,90.0,3.32,0.730000,0,1,...,1,0,0,0,0,1,0,0,0,1
4,5,4.0,7964,DPP-DTT,299.0,90.0,3.32,1.860000,0,1,...,1,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,654,7.0,7964,DPP-DTT,204.0,66.0,3.10,0.132000,1,1,...,0,0,0,1,0,0,0,0,0,1
355,655,2.0,6212,DPP-DTT,110.0,45.0,2.47,0.000256,1,1,...,0,0,0,1,0,0,0,0,0,1
359,656,6.0,6212,DPP-DTT,110.0,45.0,2.47,0.000239,1,1,...,0,0,0,1,0,0,0,0,0,1
363,657,8.0,6212,DPP-DTT,110.0,45.0,2.47,0.002700,1,1,...,0,0,0,1,0,0,0,0,0,1


In [20]:
num_rows, num_columns = unique_combined_df_DPP_DTT.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

Number of rows: 147
Number of columns: 28


In [21]:
print(unique_combined_df_DPP_DTT.isna().sum(axis=0))
print('Total observations: {}'.format(unique_combined_df_DPP_DTT.shape[0]))
print('Total features: {}'.format(unique_combined_df_DPP_DTT.shape[1]))


sample_id                             0
solution_concentration                0
solvent_pubchem_cid                   0
polymer_common_name                   0
polymer_mw                            2
polymer_mn                            3
polymer_dispersity                    3
hole_mobility                         0
solution_treatment                    0
substrate_pretreatment                0
post_process                          0
channel_width                         4
channel_length                        4
dielectric_thickness                 18
film_deposition_type_drop             0
film_deposition_type_spin             0
dielectric_material_SiO2              0
electrode_configuration_BGBC          0
electrode_configuration_BGTC          0
electrode_configuration_TGBC          0
gate_material_Other                   0
film_deposition_type_MGC              0
dielectric_material_other             0
treatment_type_sam                    0
solution_treatment_aging              0


In [23]:
unique_combined_df_DPP_DTT.to_excel('combined_df_DPP_DTT.xlsx', index=True, header=True)